<a href="https://colab.research.google.com/github/ptkoo/machineLearningJourney/blob/main/DecisionTreeMLAssignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import math
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Section 1: Example Dataset from Lecture Slides

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/car.csv')

In [ ]:
df.columns = [col.lower() for col in df.columns]
df

,type,color,tyre,doors,label
0,suv,red,white,2,1
1,car,green,black,2,1
2,car,blue,white,2,1
3,suv,green,white,4,1
4,car,red,black,2,1
5,minivan,blue,white,4,0
6,car,green,white,4,0
7,minivan,red,black,4,0
8,suv,green,black,4,0
9,suv,blue,black,2,0


In [ ]:
# num_case_dict contains the data showing the population for each label
# ie. we have 30 rows with class label1, 26 rows with label2 and 22 rows with label3
# num_case_dict = { "label1" : 30, "label2": 26, "label3": 22}

def gen_num_case_dict(df):
  # complete your code here
  labels = df.label         #Extract label from df
  counts = labels.value_counts()   #count occurances of each label
  num_case_dict = counts.to_dict()
  return num_case_dict

In [ ]:
num_case_dict = gen_num_case_dict(df)
num_case_dict

{0: 9, 1: 5}

In [ ]:
assert num_case_dict == {1: 5, 0: 9}

In [ ]:
# input num_case_array and return the entropy value

def cal_entropy(num_case_dict):
  # complete your code here
  total_data = sum(num_case_dict.values())
  entropy_sum = 0
  for label, count in num_case_dict.items():
    probability = count/total_data
    entropy_sum -= probability * math.log2(probability) # entropy = - Summation (p log2 p)

  return entropy_sum

In [ ]:
cal_entropy( num_case_dict )

0.9402859586706311

In [ ]:
assert np.allclose(cal_entropy( num_case_dict ), 0.9402859586706311)

In [ ]:
# split by color red

df_red = df[ df['color'] == 'red']

display(df_red)

num_case_dict_red = gen_num_case_dict(df_red)

print(num_case_dict_red)

entropy_red = cal_entropy(num_case_dict_red)

print('entropy_red', entropy_red)

entropy_red_weighted = (len(df_red)/len(df)) * entropy_red # Total Impurity
entropy_red_weighted

,type,color,tyre,doors,label
0,suv,red,white,2,1
4,car,red,black,2,1
7,minivan,red,black,4,0
10,suv,red,black,2,0


{1: 2, 0: 2}
entropy_red 1.0


0.2857142857142857

In [ ]:
df[df['color']=='red']

,type,color,tyre,doors,label
0,suv,red,white,2,1
4,car,red,black,2,1
7,minivan,red,black,4,0
10,suv,red,black,2,0


In [ ]:
# calculate the entropy gain from a decision tree split using a particular feature on a particular df
# entropy parent - weighted entropy child
# Expected reduction in entropy due to sorting on A

def cal_entropy_split_gain(df, split_feature):
  # complete your code here
  # Parent Entropy
  num_case = gen_num_case_dict(df)
  # print(num_case)
  parent_entropy = cal_entropy(num_case)
  # End

  # Weighted Averge* childEntropy
  split_feat = df[split_feature]
  # print(split_feat)
  feat_counts = split_feat.value_counts()
  total_data = len(df)
  child_entropy = 0


  for element, count in feat_counts.items():
    subset = df[split_feat == element]
    # print(element, subset)
    subset_weight = count/total_data
    sub_num_case = gen_num_case_dict(subset)
    # print(element, sub_num_case)
    subset_entropy = cal_entropy(sub_num_case)
    child_entropy += subset_weight * subset_entropy

  # Information gain = parent_entropy - weighted_average * child_entropy
  entropy_gain = parent_entropy - child_entropy
  # the splitting on the particular feature reduces the overall uncertainty or impurity in the dataset.

  return entropy_gain

In [ ]:
cal_entropy_split_gain(df, 'color')

0.02922256565895487

In [ ]:

assert np.allclose( cal_entropy_split_gain(df, 'color'), 0.02922256565895487 )

In [ ]:
cal_entropy_split_gain(df, 'type')

0.19996253177061118

In [ ]:
assert np.allclose( cal_entropy_split_gain(df, 'type'), 0.19996253177061118 )

In [ ]:
cal_entropy_split_gain(df, 'doors')

0.15183550136234159

In [ ]:
assert np.allclose( cal_entropy_split_gain(df, 'doors'), 0.15183550136234159 )

In [ ]:
cal_entropy_split_gain(df, 'tyre')

0.04812703040826949

In [ ]:
assert np.allclose( cal_entropy_split_gain(df, 'tyre'), 0.04812703040826949 )

In [ ]:
# Given the df and available features for conducting split, find the feature that produces maximum entropy gain.

def choose_best_split(df, available_feature_array):
  # complete your code here
  max_gain = float('-inf')   # negative infinity
  feature_with_max_entropy_gain = None

  for feature in available_feature_array:
    gain = cal_entropy_split_gain(df,feature)
    if gain > max_gain:
      max_gain = gain
      feature_with_max_entropy_gain = feature

  return feature_with_max_entropy_gain

In [ ]:
columns_list = df.columns.tolist()
columns_list.remove('label')
best_split_feature = choose_best_split(df, columns_list)
best_split_feature

'type'

In [ ]:
assert best_split_feature == 'type'

In [ ]:
# Generate a dictionary containing the split key and resulting df due to the split key and split feature.

def gen_feature_split_result(df, split_feature):
  # complete your code here
  df_split_dict = {}

  # Iterate over unique values of the feature
  for value in df[split_feature].unique():
      # Subset the DataFrame based on the current feature value
      subset = df[df[split_feature] == value].reset_index(drop=True)
      # Store the subset in the dictionary with the feature value as the key
      df_split_dict[value] = subset

  return df_split_dict

In [ ]:
df_split_dict = gen_feature_split_result(df, best_split_feature)
df_split_dict

{'suv':   type  color   tyre  doors  label
 0  suv    red  white      2      1
 1  suv  green  white      4      1
 2  suv  green  black      4      0
 3  suv   blue  black      2      0
 4  suv    red  black      2      0
 5  suv  green  black      2      0,
 'car':   type  color   tyre  doors  label
 0  car  green  black      2      1
 1  car   blue  white      2      1
 2  car    red  black      2      1
 3  car  green  white      4      0
 4  car   blue  black      4      0,
 'minivan':       type  color   tyre  doors  label
 0  minivan   blue  white      4      0
 1  minivan    red  black      4      0
 2  minivan  green  white      4      0}

In [ ]:
assert len(df_split_dict) == 3
assert len(df_split_dict['suv']) == 6
assert len(df_split_dict['minivan']) == 3
assert len(df_split_dict['car']) == 5

In [ ]:
df_split_dict['suv']

,type,color,tyre,doors,label
0,suv,red,white,2,1
1,suv,green,white,4,1
2,suv,green,black,4,0
3,suv,blue,black,2,0
4,suv,red,black,2,0
5,suv,green,black,2,0


In [ ]:
df_split_dict['minivan']

,type,color,tyre,doors,label
0,minivan,blue,white,4,0
1,minivan,red,black,4,0
2,minivan,green,white,4,0


In [ ]:
df_split_dict['car']

,type,color,tyre,doors,label
0,car,green,black,2,1
1,car,blue,white,2,1
2,car,red,black,2,1
3,car,green,white,4,0
4,car,blue,black,4,0


## Section 2: Create Tree Structure

In [ ]:
class TreeGraph:
  def __init__(self, df, available_features):
    self.root = Node(df, None, None, None, available_features)
    self.nodes = []
    self.nodes.append(self.root)

  def add_node(self, df, parent_node, parent_edge, parent_split_feature, available_features):
    new_node = Node(df, parent_node, parent_edge, parent_split_feature, available_features)
    self.nodes.append(new_node)
    if parent_node is not None:
      parent_node.add_child(  parent_edge, new_node )

  def gen_decision_tree(self, current_node = None):
    # Complete this code here
    # When you run this function it should generate the full decision tree by creating all the nodes for the tree
    if current_node is None:
      current_node = self.root

    # Base Case 1
    # Check if the node has been explored and if it's a leaf node (if unique = leaf)
    if current_node.explored or len(current_node.df['label'].unique()) == 1:
        current_node.explored = True
        return

    # Find the best split feature for the current node
    best_split_feature = choose_best_split(current_node.df, current_node.available_features)
    # print("Best Split Feature", best_split_feature)

    # Base Case 2
    # If no feature can split the data further, mark this node as explored // leaf node
    if best_split_feature is None:
        current_node.explored = True
        return

    current_node.split_feature = best_split_feature

    # Split the data based on the best feature
    unique_values = current_node.df[best_split_feature].unique()
    # print(current_node.df[best_split_feature])
    # print("Unique Values",unique_values)
    for value in unique_values:
        new_df = current_node.df[current_node.df[best_split_feature] == value]
        # print("newDF", new_df)
        new_available_features = current_node.available_features.copy()
        new_available_features.remove(best_split_feature)
        self.add_node(new_df, current_node, value, best_split_feature, new_available_features)

    # Mark the current node as explored
    current_node.explored = True

    # Recursively generate decision trees for children
    # print(current_node.children)
    for child in current_node.children.values():
        self.gen_decision_tree(child)


In [ ]:
class Node:
  def __init__(self, df, parent, parent_edge, parent_split_feature, available_features):
    self.df = df
    self.parent = parent
    self.parent_edge = parent_edge
    self.children = {}
    self.split_feature = None
    self.available_features = available_features
    self.explored = False
    self.predicted_label = None
    if parent is not None:
      parent.split_feature = parent_split_feature

  def add_child(self, split_key, new_node):
    self.children[split_key] = new_node

In [ ]:
type(df.columns)

pandas.core.indexes.base.Index

In [ ]:
columns_list = df.columns.tolist()
columns_list.remove('label')
g = TreeGraph(df, columns_list)
g.gen_decision_tree()
g.nodes

In [ ]:
assert len(g.nodes) == 8

In [ ]:
assert g.root.split_feature == 'type'

In [ ]:
assert len(g.root.children) == 3

In [ ]:
g.root.children.items()

dict_items([('suv', <__main__.Node object at 0x7b1104755c30>), ('car', <__main__.Node object at 0x7b1104755cc0>), ('minivan', <__main__.Node object at 0x7b1104755ae0>)])

In [ ]:
for node in g.nodes:
  if node.split_feature == 'doors':
    test_case = node.df
    display(test_case)

#assert len(test_case) == 5
assert np.array_equal( test_case['color'].to_numpy() , np.array(['green', 'blue', 'red', 'green', 'blue'], dtype=object) )

,type,color,tyre,doors,label
1,car,green,black,2,1
2,car,blue,white,2,1
4,car,red,black,2,1
6,car,green,white,4,0
11,car,blue,black,4,0


In [ ]:
for node in g.nodes:
  if node.split_feature == 'tyre':
    test_case = node.df
    display(test_case)

assert len(test_case) == 6
assert np.array_equal( test_case['color'].to_numpy() , np.array(['red', 'green', 'green', 'blue', 'red', 'green'], dtype=object) )

,type,color,tyre,doors,label
0,suv,red,white,2,1
3,suv,green,white,4,1
8,suv,green,black,4,0
9,suv,blue,black,2,0
10,suv,red,black,2,0
12,suv,green,black,2,0


In [ ]:
for node in g.nodes:
  if node.split_feature == 'type':
    test_case = node.df
    display(test_case)

assert len(test_case) == 14
assert np.array_equal( test_case['color'].to_numpy() , np.array(['red', 'green', 'blue', 'green', 'red', 'blue', 'green', 'red', 'green', 'blue', 'red', 'blue', 'green', 'green'], dtype=object) )

,type,color,tyre,doors,label
0,suv,red,white,2,1
1,car,green,black,2,1
2,car,blue,white,2,1
3,suv,green,white,4,1
4,car,red,black,2,1
5,minivan,blue,white,4,0
6,car,green,white,4,0
7,minivan,red,black,4,0
8,suv,green,black,4,0
9,suv,blue,black,2,0


## Section 3: Do Inference to Make Predictions

In [ ]:
# Now let's do inference with the tree we've generated

In [ ]:
df

,type,color,tyre,doors,label
0,suv,red,white,2,1
1,car,green,black,2,1
2,car,blue,white,2,1
3,suv,green,white,4,1
4,car,red,black,2,1
5,minivan,blue,white,4,0
6,car,green,white,4,0
7,minivan,red,black,4,0
8,suv,green,black,4,0
9,suv,blue,black,2,0


In [ ]:
import copy

df_prediction = df.copy()
df_prediction['prediction'] = None

In [ ]:
df_prediction

,type,color,tyre,doors,label,prediction
0,suv,red,white,2,1,None
1,car,green,black,2,1,None
2,car,blue,white,2,1,None
3,suv,green,white,4,1,None
4,car,red,black,2,1,None
5,minivan,blue,white,4,0,None
6,car,green,white,4,0,None
7,minivan,red,black,4,0,None
8,suv,green,black,4,0,None
9,suv,blue,black,2,0,None


In [ ]:
# This function should fill in the predicted values using our decision tree into the df in the column 'prediction'

def generate_prediction(df_prediction, treeGraph):
  # Complete the code here
  for index, row in df_prediction.iterrows():   #row interation - Return index of row and data of row
    current_node = treeGraph.root # <<<
    while current_node.children:
      split_feature_value = row[current_node.split_feature]
      if split_feature_value in current_node.children:
          current_node = current_node.children[split_feature_value] # <<<
      else:
          # If the value is not found in children, we cannot predict, so breaking the loop
          break
    # Assign the predicted label to the 'prediction' column
    df_prediction.at[index, 'prediction'] = current_node.df['label'].mode()[0]

  return df_prediction
 # traverses down the decision tree for each row and assigns the predicted label obtained from the leaf node reached during traversal.

In [ ]:
prediction_result_df = generate_prediction(df_prediction,g)
prediction_result_df

,type,color,tyre,doors,label,prediction
0,suv,red,white,2,1,1
1,car,green,black,2,1,1
2,car,blue,white,2,1,1
3,suv,green,white,4,1,1
4,car,red,black,2,1,1
5,minivan,blue,white,4,0,0
6,car,green,white,4,0,0
7,minivan,red,black,4,0,0
8,suv,green,black,4,0,0
9,suv,blue,black,2,0,0


In [ ]:
assert np.array_equal( prediction_result_df['prediction'].to_numpy(), np.array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=object) )

In [ ]:
def prediction_accuracy(prediction_result_df):
  correct = 0
  for index, row in prediction_result_df.iterrows():
    if row['label'] == row['prediction']:
      correct = correct + 1
  accuracy = correct / len(prediction_result_df)
  return accuracy

In [ ]:
prediction_accuracy_result = prediction_accuracy(prediction_result_df)
prediction_accuracy_result

1.0

In [ ]:
assert prediction_accuracy_result == 1

## Section 4: Test with Proper Dataset

In [ ]:
# Get dataset from here: https://archive.ics.uci.edu/dataset/19/car+evaluation

# buying - buying price
# maintainance - price of the maintenance
# doors - number of doors
# person - capacity in terms of persons to carry
# luggage_boot - the size of luggage boot
# safety - estimated safety of the car
# label - evaulation level (unacceptable, acceptable, good, very good)

df_car = pd.read_csv('/content/drive/MyDrive/car.data', sep=',', header=None)

# Assign column names to the DataFrame
df_car.columns = ['buying', 'maintainance', 'doors', 'person', 'luggage_boot', 'safety', 'label' ]

df_car.head()

,buying,maintainance,doors,person,luggage_boot,safety,label
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [ ]:
from sklearn.model_selection import train_test_split

df_car_train, df_car_test = train_test_split(df_car, test_size=0.2, random_state=42)

In [ ]:
columns_list = df_car.columns.tolist()
columns_list.remove('label')
g_car = TreeGraph(df_car_train, columns_list)
g_car.gen_decision_tree()

In [ ]:
len(g_car.nodes)

349

In [ ]:
df_car_prediction = df_car_test.copy()
df_car_prediction['prediction'] = None
df_car_prediction_result = generate_prediction(df_car_prediction, g_car)

In [ ]:
df_car_prediction_result['prediction']

599     unacc
1201     good
628     unacc
1498      acc
1263    unacc
        ...  
100     unacc
274     unacc
1206    unacc
101     unacc
1084    unacc
Name: prediction, Length: 346, dtype: object

In [ ]:
df_car_prediction_result

,buying,maintainance,doors,person,luggage_boot,safety,label,prediction
599,high,high,4,2,med,high,unacc,unacc
1201,med,low,2,4,med,med,acc,good
628,high,high,5more,2,big,med,unacc,unacc
1498,low,high,5more,4,med,med,acc,acc
1263,med,low,4,more,med,low,unacc,unacc
...,...,...,...,...,...,...,...,...
100,vhigh,vhigh,5more,more,small,med,unacc,unacc
274,vhigh,med,4,2,med,med,unacc,unacc
1206,med,low,2,more,small,low,unacc,unacc
101,vhigh,vhigh,5more,more,small,high,unacc,unacc


In [ ]:
prediction_accuracy_result = prediction_accuracy(df_car_prediction_result)
prediction_accuracy_result

0.9248554913294798

In [ ]:
assert prediction_accuracy_result > 0.9

In [ ]:
assert len(df_car_prediction_result[df_car_prediction_result['prediction'].isna()]) == 0